In [1]:
import matplotlib.pyplot as plt;
import pandas as pd
import numpy as np
from sklearn import preprocessing
from kneed import DataGenerator, KneeLocator
from sklearn.metrics import accuracy_score
from sklearn import metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import BernoulliNB

In [2]:
trainSize = 400000
testSize = 50000
size = trainSize+testSize # 400000 for taining and 50000 for testing
data = pd.read_csv("mixed.csv", nrows=size)
data = data.drop(columns="No.")
columns_titles = ["Time", "Source", "Destination", "Length", "Info", "Protocol"]
data = data.reindex(columns=columns_titles)
label_names = np.unique(data.iloc[:,5])

le = preprocessing.LabelEncoder()
data.iloc[:,1] = le.fit_transform(data.iloc[:,1])
data.iloc[:,2] = le.fit_transform(data.iloc[:,2])
data.iloc[:,4] = le.fit_transform(data.iloc[:,4])
data.iloc[:,5] = le.fit_transform(data.iloc[:,5])

def chooseObsMan(dic_f):
    chosenIDs=np.array(())
    for k,d in dic_f.items():
        temp=np.where(y_train==k)[0]
        if temp.size < dic_f[k]:
            temp = dic_f[k]
        z=np.random.choice(temp,dic_f[k],replace=False)
        chosenIDs=np.concatenate((chosenIDs,z))
    return np.int64(chosenIDs)
def label_index(row,threshold=0):
    
    if threshold==0:
        threshold=np.max(row)
    temp_i=np.where(row>=threshold)[0]
    if temp_i.shape[0]>0:
        return temp_i[0]+1
    else:
        return -1

y_train = data.iloc[:trainSize,5]
x_train = data.iloc[:trainSize,:-1]
#x_train = preprocessing.normalize(data.iloc[:trainSize,:-1])

y_test = data.iloc[trainSize:,5]
x_test = data.iloc[trainSize:,:-1]
#x_test = preprocessing.normalize(data.iloc[trainSize:,:-1])

li = {}
r = int(size/100000 + 1)*30
for i in np.unique(data.iloc[:,-1]):
    if data['Protocol'].value_counts()[i] < r:
        li.update({i : data['Protocol'].value_counts()[i]})    
    else :
        li.update({i : r})

labelled=np.zeros(x_train.shape[0])
labelled[:]=-1
labelledIndex=chooseObsMan(li)
labelled[labelledIndex]=y_train[labelledIndex]

In [3]:
clf = BernoulliNB()
clf.fit(x_train, y_train)

BernoulliNB()

In [4]:
predicted = clf.predict(x_train)

In [5]:
print((predicted == y_train).mean())

0.6544225


In [9]:
print("------------------------------------Paper 2---------------------------------------")

print('\n*********Training*********\n')

matrices = metrics.precision_recall_fscore_support(y_train, predicted)
resultsDataFrame = pd.DataFrame()
for i in range(0,len(matrices)):
    temp = pd.DataFrame([matrices[i]],columns=label_names)
    resultsDataFrame = resultsDataFrame.append(temp, ignore_index=True)
resultsDataFrame.insert(0,'  ',['Precision','Recall','Fscore','Support'])
print(resultsDataFrame)

------------------------------------Paper 2---------------------------------------

*********Training*********

                 ARP  BROWSER       DHCP  DHCPv6     DNS      FTP  FTP-DATA  \
0  Precision     0.0      0.0   1.000000     0.0     0.0      0.0       0.0   
1     Recall     0.0      0.0   0.028571     0.0     0.0      0.0       0.0   
2     Fscore     0.0      0.0   0.055556     0.0     0.0      0.0       0.0   
3    Support  5363.0   1190.0  35.000000   329.0  8390.0  28991.0    1604.0   

      HTTP  HTTP/XML  ...    NBNS    NBSS  NTP  OCSP  PKIX-CRL     POP  \
0      0.0       0.0  ...     0.0     0.0  0.0   0.0       0.0     0.0   
1      0.0       0.0  ...     0.0     0.0  0.0   0.0       0.0     0.0   
2      0.0       0.0  ...     0.0     0.0  0.0   0.0       0.0     0.0   
3  33507.0      91.0  ...  1442.0  1110.0  4.0   1.0       2.0  7497.0   

      SMB   SMTP            TCP   TLSv1  
0     0.0    0.0       0.654332     0.0  
1     0.0    0.0       1.000000     0

In [7]:
correctLabelled=metrics.accuracy_score(y_train, predicted,normalize=False)
incorrectLabelled=sum(resultsDataFrame.iloc[3,1:]) - correctLabelled
print([correctLabelled,incorrectLabelled])

[261769, 138231.0]


In [10]:
numbers=resultsDataFrame.iloc[3,1:]
correctLabelledForEachclass=resultsDataFrame.iloc[3,1:] *resultsDataFrame.iloc[1,1:]
incorrectLabelledForEachclass=resultsDataFrame.iloc[3,1:]-correctLabelledForEachclass
results1 = pd.DataFrame(columns = ['Total','Correct', 'Incorrect'])
results1['Total'] = numbers
results1['Correct'] = correctLabelledForEachclass
results1['Incorrect'] = incorrectLabelledForEachclass
print(results1)


           Total Correct Incorrect
ARP         5363       0      5363
BROWSER     1190       0      1190
DHCP          35       1        34
DHCPv6       329       0       329
DNS         8390       0      8390
FTP        28991       0     28991
FTP-DATA    1604       0      1604
HTTP       33507       0     33507
HTTP/XML      91       0        91
ICMP           1       0         1
ICMPv6        28       0        28
IGMPv3        30       0        30
IMAP        6148       0      6148
IMAP/IMF     103     103         0
LANMAN      1172       0      1172
LLMNR         28       0        28
MDNS       28837       1     28836
NBNS        1442       0      1442
NBSS        1110       0      1110
NTP            4       0         4
OCSP           1       0         1
PKIX-CRL       2       0         2
POP         7497       0      7497
SMB         6970       0      6970
SMTP         630       0       630
TCP       261664  261664         0
TLSv1       4833       0      4833
